In [1]:
# this script is intended to help generate install_dependecy scripts for different platforms


In [2]:
import os
import glob
import json
import datetime

In [70]:
def bash_header(install_prefix='/opt', workdir='/tmp'):
    current_year = datetime.datetime.now().year
    current_date = str(datetime.datetime.now())
    return '''#!/usr/bin/env bash
# (c) Tuplex team 2017-{}
# auto-generated on {}
# install all dependencies required to compile tuplex + whatever is needed for profiling
# everything will be installed to {} by default'''.format(current_year, current_date, install_prefix) + \
'\n' + '''
# need to run this with root privileges
if [[ $(id -u) -ne 0 ]]; then
  echo "Please run this script with root privileges"
  exit 1
fi

export DEBIAN_FRONTEND=noninteractive

PREFIX=${{PREFIX:-{}}}
WORKDIR=${{WORKDIR:-{}}}

echo ">> Installing packages into ${{PREFIX}}"
mkdir -p $PREFIX && chmod 0755 $PREFIX
mkdir -p $PREFIX/sbin
mkdir -p $PREFIX/bin
mkdir -p $PREFIX/share
mkdir -p $PREFIX/include
mkdir -p $PREFIX/lib

echo ">> Files will be downloaded to ${{WORKDIR}}/tuplex-downloads"
WORKDIR=$WORKDIR/tuplex-downloads
mkdir -p $WORKDIR

PYTHON_EXECUTABLE=${{PYTHON_EXECUTABLE:-python3}}
PYTHON_BASENAME="$(basename -- $PYTHON_EXECUTABLE)"
PYTHON_VERSION=$(${{PYTHON_EXECUTABLE}} --version)
echo ">> Building dependencies for ${{PYTHON_VERSION}}"

'''.format(install_prefix, workdir)

In [71]:
with open('test.sh', 'w') as fp:
    fp.write(bash_header())

In [72]:
print(bash_header())

#!/usr/bin/env bash
# (c) Tuplex team 2017-2022
# auto-generated on 2022-08-12 11:00:30.688359
# install all dependencies required to compile tuplex + whatever is needed for profiling
# everything will be installed to /opt by default

# need to run this with root privileges
if [[ $(id -u) -ne 0 ]]; then
  echo "Please run this script with root privileges"
  exit 1
fi

export DEBIAN_FRONTEND=noninteractive

PREFIX=${PREFIX:-/opt}
WORKDIR=${WORKDIR:-/tmp}

echo ">> Installing packages into ${PREFIX}"
mkdir -p $PREFIX && chmod 0755 $PREFIX
mkdir -p $PREFIX/sbin
mkdir -p $PREFIX/bin
mkdir -p $PREFIX/share
mkdir -p $PREFIX/include
mkdir -p $PREFIX/lib

echo ">> Files will be downloaded to ${WORKDIR}/tuplex-downloads"
WORKDIR=$WORKDIR/tuplex-downloads
mkdir -p $WORKDIR

PYTHON_EXECUTABLE=${PYTHON_EXECUTABLE:-python3}
PYTHON_BASENAME="$(basename -- $PYTHON_EXECUTABLE)"
PYTHON_VERSION=$(${PYTHON_EXECUTABLE} --version)
echo ">> Building dependencies for ${PYTHON_VERSION}"




In [103]:
# configure here which versions to use for each platform

CMAKE_VERSION='3.23.2'
YAMLCPP_VERSION='0.6.3'
CELERO_VERSION='2.6.0'
ANTLR_VERSION='4.8'
AWSSDK_VERSION='1.9.133'
AWSLAMBDACPP_VERSION='0.2.6'
PCRE2_VERSION='10.39'
PROTOBUF_VERSION='3.12'
CLOUDPICKLE_VERSION='<2.0.0'
BOOST_VERSION='1.75.0'
GCC_VERSION='10'
LLVM_VERSION='9.0.1'

WORKDIR='/tmp'
PREFIX='/opt'

# derived vars
GCC_VERSION_MAJOR=GCC_VERSION.split('.')[0]
CMAKE_VERSION_MAJOR,CMAKE_VERSION_MINOR,CMAKE_VERSION_PATCH=tuple(CMAKE_VERSION.split('.'))

In [104]:
def apt_dependencies(osname='ubuntu:22.04'):
    
    packages_dict = {'ubuntu:22.04': '''apt-utils curl libxml2-dev vim build-essential libssl-dev zlib1g-dev libncurses5-dev \\
    libncursesw5-dev libreadline-dev libsqlite3-dev libgdbm-dev libdb5.3-dev \\
    libbz2-dev libexpat1-dev liblzma-dev tk-dev libffi-dev wget git''',
                     'ubuntu:20.04': '''software-properties-common curl build-essential wget git libedit-dev libz-dev \\
                   python3-yaml python3-pip pkg-config libssl-dev libcurl4-openssl-dev curl \\
                   uuid-dev libffi-dev libmagic-dev \\
                   doxygen doxygen-doc doxygen-latex doxygen-gui graphviz \\
                   libgflags-dev libncurses-dev \\
                   openjdk-8-jdk libyaml-dev ninja-build gcc-{} g++-{} autoconf libtool m4
                     '''.format(GCC_VERSION_MAJOR, GCC_VERSION_MAJOR),
                    'ubuntu:18.04': '''build-essential libxml2-dev
 autoconf curl automake libtool software-properties-common wget libedit-dev libz-dev \\
  python3-yaml pkg-config libssl-dev libcurl4-openssl-dev curl \\
  uuid-dev git python3.7 python3.7-dev python3-pip libffi-dev \\
  doxygen doxygen-doc doxygen-latex doxygen-gui graphviz \\
  gcc-{} g++-{} libgflags-dev libncurses-dev \\
  awscli openjdk-8-jdk libyaml-dev libmagic-dev ninja-build
                    '''.format(GCC_VERSION_MAJOR, GCC_VERSION_MAJOR)}
    
    return 'apt update -y\n' + \
'''
apt-get install -y {}
  
ldconfig
export CC=gcc-{}
export CXX=g++-{}
'''.format(packages_dict[osname], GCC_VERSION_MAJOR, GCC_VERSION_MAJOR)

In [105]:
print(apt_dependencies())

apt update -y

apt-get install -y apt-utils curl libxml2-dev vim build-essential libssl-dev zlib1g-dev libncurses5-dev \
    libncursesw5-dev libreadline-dev libsqlite3-dev libgdbm-dev libdb5.3-dev \
    libbz2-dev libexpat1-dev liblzma-dev tk-dev libffi-dev wget git
  
ldconfig
export CC=gcc-10
export CXX=g++-10



In [106]:
def install_cmake():
    return '''
# fetch recent cmake & install
CMAKE_VER_MAJOR={}
CMAKE_VER_MINOR={}
CMAKE_VER_PATCH={}'''.format(CMAKE_VERSION_MAJOR,CMAKE_VERSION_MINOR,CMAKE_VERSION_PATCH) + '''
CMAKE_VER="${CMAKE_VER_MAJOR}.${CMAKE_VER_MINOR}"
CMAKE_VERSION="${CMAKE_VER}.${CMAKE_VER_PATCH}"
URL=https://github.com/Kitware/CMake/releases/download/v${CMAKE_VERSION}/cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz
mkdir -p ${WORKDIR}/cmake && cd ${WORKDIR}/cmake &&
  curl -sSL $URL -o cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  tar -v -zxf cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  rm -f cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  cd cmake-${CMAKE_VERSION}-linux-x86_64 &&
  cp -rp bin/* ${PREFIX}/bin/ &&
  cp -rp share/* ${PREFIX}/share/ &&
  cd / && rm -rf ${WORKDIR}/cmake
    '''

In [107]:
print(install_cmake())


# fetch recent cmake & install
CMAKE_VER_MAJOR=3
CMAKE_VER_MINOR=23
CMAKE_VER_PATCH=2
CMAKE_VER="${CMAKE_VER_MAJOR}.${CMAKE_VER_MINOR}"
CMAKE_VERSION="${CMAKE_VER}.${CMAKE_VER_PATCH}"
URL=https://github.com/Kitware/CMake/releases/download/v${CMAKE_VERSION}/cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz
mkdir -p ${WORKDIR}/cmake && cd ${WORKDIR}/cmake &&
  curl -sSL $URL -o cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  tar -v -zxf cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  rm -f cmake-${CMAKE_VERSION}-linux-x86_64.tar.gz &&
  cd cmake-${CMAKE_VERSION}-linux-x86_64 &&
  cp -rp bin/* ${PREFIX}/bin/ &&
  cp -rp share/* ${PREFIX}/share/ &&
  cd / && rm -rf ${WORKDIR}/cmake
    


In [108]:
def install_boost():
    # may need to include the following to work for boost python
    # # fix up for boost python a link
    # cd /usr/local/include/ && ln -s python3.6m python3.6 && cd - || exit
    
    boost_reformatted = BOOST_VERSION.replace('.', '_')
    
    return '''    
mkdir -p ${WORKDIR/boost}
    
# build incl. boost python
pushd ${WORKDIR/boost} && ''' + \
'wget https://boostorg.jfrog.io/artifactory/main/release/{}/source/boost_{}.tar.gz && tar xf boost_{}.tar.gz'.format(BOOST_VERSION, boost_reformatted, boost_reformatted, boost_reformatted) + \
' && cd ${{WORKDIR/boost}}/boost_{}'.format(boost_reformatted) + \
''' \\
&& ./bootstrap.sh --with-python=${PYTHON_EXECUTABLE} --prefix=${PREFIX} --with-libraries="thread,iostreams,regex,system,filesystem,python,stacktrace,atomic,chrono,date_time" \\
 && ./b2 cxxflags="-fPIC" link=static -j "$(nproc)" \\
 && ./b2 cxxflags="-fPIC" link=static install'''

In [109]:
print(install_boost())

    
mkdir -p ${WORKDIR/boost}
    
# build incl. boost python
pushd ${WORKDIR/boost} && wget https://boostorg.jfrog.io/artifactory/main/release/1.75.0/source/boost_1_75_0.tar.gz && tar xf boost_1_75_0.tar.gz && cd ${WORKDIR/boost}/boost_1_75_0 \
&& ./bootstrap.sh --with-python=${PYTHON_EXECUTABLE} --prefix=${PREFIX} --with-libraries="thread,iostreams,regex,system,filesystem,python,stacktrace,atomic,chrono,date_time" \
 && ./b2 cxxflags="-fPIC" link=static -j "$(nproc)" \
 && ./b2 cxxflags="-fPIC" link=static install


In [141]:
def install_llvm():
    
    LLVM_VERSION_MAJOR=LLVM_VERSION.split('.')[0]
    workdir='${WORKDIR}/llvm'
    code = 'mkdir -p {} && cd {} && '.format(workdir, workdir)  + \
'''wget https://github.com/llvm/llvm-project/releases/download/llvmorg-{version}/llvm-{version}.src.tar.xz \\ 
&& wget https://github.com/llvm/llvm-project/releases/download/llvmorg-{version}/clang-{version}.src.tar.xz \\ 
&& tar xf llvm-{version}.src.tar.xz && tar xf clang-{version}.src.tar.xz \\ '''.format(version=LLVM_VERSION)

    code += '''\n&& mkdir llvm{maj} && mv clang-{version}.src llvm{maj}/clang \\
&& mv llvm-{version}.src llvm{maj}/llvm-{version}.src \\
&& cd llvm{maj}'''.format(version=LLVM_VERSION, maj=LLVM_VERSION_MAJOR)

    code += ''' && mkdir build && cd build \\
&& cmake -DLLVM_ENABLE_RTTI=ON -DLLVM_ENABLE_EH=ON \\
        -DLLVM_ENABLE_PROJECTS="clang" \\
         -DLLVM_TARGETS_TO_BUILD="X86" -DCMAKE_BUILD_TYPE=Release \\
         -DCMAKE_INSTALL_PREFIX=/opt/llvm-{version} ../llvm-{version}.src \\
&& make -j "$(nproc)" && make install'''.format(version=LLVM_VERSION)
    
    return code

In [142]:
print(install_llvm())

mkdir -p ${WORKDIR}/llvm && cd ${WORKDIR}/llvm && wget https://github.com/llvm/llvm-project/releases/download/llvmorg-9.0.1/llvm-9.0.1.src.tar.xz \ 
&& wget https://github.com/llvm/llvm-project/releases/download/llvmorg-9.0.1/clang-9.0.1.src.tar.xz \ 
&& tar xf llvm-9.0.1.src.tar.xz && tar xf clang-9.0.1.src.tar.xz \ 
&& mkdir llvm9 && mv clang-9.0.1.src llvm9/clang \
&& mv llvm-9.0.1.src llvm9/llvm-9.0.1.src \
&& cd llvm9 && mkdir build && cd build \
&& cmake -DLLVM_ENABLE_RTTI=ON -DLLVM_ENABLE_EH=ON \
        -DLLVM_ENABLE_PROJECTS="clang" \
         -DLLVM_TARGETS_TO_BUILD="X86" -DCMAKE_BUILD_TYPE=Release \
         -DCMAKE_INSTALL_PREFIX=/opt/llvm-9.0.1 ../llvm-9.0.1.src \
&& make -j "$(nproc)" && make install
